In [50]:
import pandas as pd
import numpy as np
import datetime

In [51]:
!pwd

/Users/joshua/Dropbox/Desktop/NAFTA


In [98]:
%cd /Users/joshua/Desktop/NAFTA

/Users/joshua/Dropbox/Desktop/NAFTA


In [99]:
!ls

Originals      inventory.xlsx	     potran_01012017.xlsx  sotran_01012017.xlsx
alt_item.xlsx  pomast_01012017.xlsx  somast_01012017.xlsx  vendors.xlsx


## 4 Questions
1. What parts did we sell in the year?
2. Which of these were to the cusotmer in question?
3. Where did we buy those parts?
4. What country are those vendors in?

## Put in data terms
1. so_line by date
2. Filter by customer
3. Join to po_mast (for vendor, possibly from po_mast to vendor is neeeded)
4. vendor country

### 1. What parts did we sell in the year

In [21]:
items_sold = pd.read_excel('sotran_01012017.xlsx')

In [26]:
items_sold.shape

(23229, 22)

In [27]:
items_sold.describe()

,custno,acustno,cost,price,qtyord,qtyshp,qtylft,qtybo,extprice,shipqty,seq,qtytoshp
count,23229.000000,2693.000000,23229.000000,23229.000000,2.322900e+04,23229.000000,2.322900e+04,23229.0,23229.000000,23229.0,23229.000000,23229.0
mean,26252.659693,44847.225771,2.027548,2.768720,3.307901e+03,2918.936502,3.889644e+02,0.0,611.640587,0.0,16.843601,0.0
std,36777.374705,80290.795870,35.896063,40.326782,2.285979e+04,14132.391956,1.742247e+04,0.0,5026.744653,0.0,20.673123,0.0
min,60.000000,200.000000,-0.001010,-1008.024000,-6.000000e+04,-60000.000000,0.000000e+00,0.0,-90431.880000,0.0,1.000000,0.0
25%,14700.000000,5315.000000,0.011130,0.031600,1.500000e+02,120.000000,0.000000e+00,0.0,25.000000,0.0,1.000000,0.0
50%,18700.000000,13800.000000,0.042370,0.100000,5.000000e+02,500.000000,0.000000e+00,0.0,76.500000,0.0,8.000000,0.0
75%,18700.000000,13911.000000,0.218000,0.485000,2.000000e+03,2000.000000,0.000000e+00,0.0,232.900000,0.0,25.000000,0.0
max,148000.000000,325700.000000,2950.000000,2950.000000,1.823000e+06,499200.000000,1.823000e+06,0.0,247901.080000,0.0,119.000000,0.0


In [28]:
items_sold.head()

,sono,custno,cname,custpo,item,altitem,acustno,acustpo,acustname,cost,...,qtyshp,qtylft,qtybo,extprice,shipqty,seq,sostat,sotype,created,qtytoshp
0,174021,800,ANDERSON INSTRUMENT CO.,PO00129032,36240V1507,36240V1507,NaN,NaN,NaN,0.07000,...,150,0,0,49.68,0,1,C,O,2017-01-03,0
1,174022,800,ANDERSON INSTRUMENT CO.,PO00129330,228305020-K1,4518400000,NaN,NaN,NaN,0.00810,...,500,0,0,20.55,0,1,C,O,2017-01-03,0
2,174016,141200,"PAR 4 PLASTICS, INC.",30296,202537,700057,NaN,NaN,NaN,0.00971,...,10000,0,0,210.00,0,1,C,O,2017-01-02,0
3,174025,145700,IN PRECISION LINK,ASSEMBLY,17006-1K0-31P,501014,NaN,NaN,NaN,0.06029,...,25000,0,0,25.00,0,1,C,O,2017-01-04,0
4,174015,18700,WELCH ALLYN/ ACCTS PAYABLE,55002058,N-1301-192-0,740027-1,NaN,NaN,NaN,0.08500,...,100,0,0,6.24,0,1,C,O,2017-01-01,0


In [53]:
pd.to_datetime(items_sold['created'])

0       2017-01-03
1       2017-01-03
2       2017-01-02
3       2017-01-04
4       2017-01-01
5       2017-01-01
6       2017-01-01
7       2017-01-01
8       2017-01-01
9       2017-01-01
10      2017-01-03
11      2017-01-03
12      2017-01-03
13      2017-01-03
14      2017-01-03
15      2017-01-03
16      2017-01-03
17      2017-01-03
18      2017-01-03
19      2017-01-03
20      2017-01-03
21      2017-01-03
22      2017-01-03
23      2017-01-03
24      2017-01-03
25      2017-01-03
26      2017-01-03
27      2017-01-03
28      2017-01-03
29      2017-01-03
           ...    
23199   2018-11-10
23200   2018-11-10
23201   2018-11-10
23202   2018-11-10
23203   2018-11-10
23204   2018-11-10
23205   2018-11-10
23206   2018-11-10
23207   2018-11-10
23208   2018-11-10
23209   2018-11-10
23210   2018-11-10
23211   2018-11-10
23212   2018-11-10
23213   2018-11-10
23214   2018-11-10
23215   2018-11-10
23216   2018-11-10
23217   2018-11-10
23218   2018-11-10
23219   2018-11-10
23220   2018

In [66]:
sales_2018 = items_sold[(items_sold['created'] >= '2018-01-01') & (items_sold['created'] <= '2018-12-31')]

In [68]:
sales_2018.head()

,sono,custno,cname,custpo,item,altitem,acustno,acustpo,acustname,cost,...,qtyshp,qtylft,qtybo,extprice,shipqty,seq,sostat,sotype,created,qtytoshp
12333,190013,117500,ANSEN CORPORATION,P46445VMI,SEM-SHCS-SEAL-L,SEM-SHCS-SEAL-L,NaN,NaN,NaN,0.52000,...,7000,0,0,3640.00,0,1,C,O,2018-01-03,0
12334,190013,117500,ANSEN CORPORATION,P46445VMI,33016-060-40,33016-060-40,NaN,NaN,NaN,0.33201,...,1550,0,0,1317.50,0,2,C,O,2018-01-03,0
12335,190013,117500,ANSEN CORPORATION,P46445VMI,20044-030-40,20044-030-40,NaN,NaN,NaN,0.08649,...,3298,0,0,388.17,0,3,C,O,2018-01-03,0
12336,190013,117500,ANSEN CORPORATION,P46445VMI,SEM-SHCS-SEAL-C,NaN,NaN,NaN,NaN,0.42000,...,500,0,0,210.00,0,4,C,O,2018-01-03,0
12337,190013,117500,ANSEN CORPORATION,P46445VMI,83012-TFP-40,83012-TFP-40,NaN,NaN,NaN,0.02323,...,34000,0,0,1904.00,0,5,C,O,2018-01-03,0


In [67]:
sales_2018.tail()

,sono,custno,cname,custpo,item,altitem,acustno,acustpo,acustname,cost,...,qtyshp,qtylft,qtybo,extprice,shipqty,seq,sostat,sotype,created,qtytoshp
23224,193302,18700,WELCH ALLYN/ ACCTS PAYABLE,55002058,4G0A1-15C-12I,721455,NaN,NaN,NaN,0.01324,...,0,5000,0,142.50,0,47,A,O,2018-11-10,0
23225,193302,18700,WELCH ALLYN/ ACCTS PAYABLE,55002058,703355,703355,NaN,NaN,NaN,1.29000,...,0,3000,0,5100.00,0,48,A,O,2018-11-10,0
23226,193302,18700,WELCH ALLYN/ ACCTS PAYABLE,55002058,17005-05C-12V,703410,NaN,NaN,NaN,0.00000,...,0,100,0,23.25,0,49,A,O,2018-11-10,0
23227,193303,800,ANDERSON INSTRUMENT CO.,146744,B0343-019,36600Q0019,NaN,NaN,NaN,0.02150,...,0,1000,0,34.50,0,1,A,O,2018-11-11,0
23228,193304,800,ANDERSON INSTRUMENT CO.,PO00133498,300-200A,300-200A,NaN,NaN,NaN,0.00000,...,0,15,0,59.25,0,1,A,O,2018-11-11,0


In [85]:
items_sold_2018 = sales_2018.drop_duplicates(['item'],keep='first', inplace=False)

In [86]:
items_sold_2018.head()

,sono,custno,cname,custpo,item,altitem,acustno,acustpo,acustname,cost,...,qtyshp,qtylft,qtybo,extprice,shipqty,seq,sostat,sotype,created,qtytoshp
12333,190013,117500,ANSEN CORPORATION,P46445VMI,SEM-SHCS-SEAL-L,SEM-SHCS-SEAL-L,NaN,NaN,NaN,0.52000,...,7000,0,0,3640.00,0,1,C,O,2018-01-03,0
12334,190013,117500,ANSEN CORPORATION,P46445VMI,33016-060-40,33016-060-40,NaN,NaN,NaN,0.33201,...,1550,0,0,1317.50,0,2,C,O,2018-01-03,0
12335,190013,117500,ANSEN CORPORATION,P46445VMI,20044-030-40,20044-030-40,NaN,NaN,NaN,0.08649,...,3298,0,0,388.17,0,3,C,O,2018-01-03,0
12336,190013,117500,ANSEN CORPORATION,P46445VMI,SEM-SHCS-SEAL-C,NaN,NaN,NaN,NaN,0.42000,...,500,0,0,210.00,0,4,C,O,2018-01-03,0
12337,190013,117500,ANSEN CORPORATION,P46445VMI,83012-TFP-40,83012-TFP-40,NaN,NaN,NaN,0.02323,...,34000,0,0,1904.00,0,5,C,O,2018-01-03,0


In [87]:
items_sold_2018.shape

(2047, 22)

In [88]:
sales_2018.item.nunique()

2047

In [90]:
nafta_items_2018 = sales_2018[['custno','cname','item','altitem']]

### 2. Which were sold to the customer in question?

In [91]:
nafta_items_2018.cname.unique()

array(['ANSEN CORPORATION', 'EJ USA, INC', 'WELCH ALLYN/ ACCTS PAYABLE',
       'ANDERSON INSTRUMENT CO.', 'COURSER INC.',
       'RTD EMBEDDED TECHNOLOGIES, INC.', 'YOUNG-FRANKLIN TOOL WORKS',
       'PREMIER DIE CASTING COMPANY', 'SCIENTIFIC SYSTEMS',
       'CASHI COMPONENTS', 'REMINGTON ARMS', 'TEXTRON LYCOMING',
       'TIMES MICROWAVE SYSTEMS', 'PARKER HANNIFIN CORPORATION',
       'ADVANCED MOTORS AND DRIVES INC.', 'GREENE TECHNOLOGIES',
       'EATON IND. MFG GMBH LEDGER 5179',
       'TRIUMPH ENGINE CONTROL SYSTEMS, LLC', 'AUDIO SEARS CORP',
       'UPSCO, INC.', 'AJEDREZ DE MONTERREY', 'RLC ELECTRONICS',
       'IRBY UTILITY SUPPLY', 'TACTAIR FLUID CONTROLS INC.',
       'SCIENTIFIC TOOL CO', 'ESPEY MFG & ELECTRONICS CORP.',
       'PELCO COMPONENT TECHNOLOGIES', 'ATRONIX',
       '4M PRECISION INDUSTRIES INC.', 'TENNYSON MACHINE',
       'ALLIANCE PRECISION PLASTICS', 'BASIC RUBBER & PLASTICS  CO.',
       'EMPIRE FIBREGLASS PRODUCTS, INC.', 'CHOSEN TECHNOLOGIES PTE LTD',
  

In [96]:
nidec_sales_2018 = nafta_items_2018[nafta_items_2018['cname'].str.contains('ADVANCED MOTORS')]

In [97]:
nidec_sales_2018.shape

(954, 4)

### 3. Where did we buy those items?

In [107]:
po_lines = pd.read_excel('potran_01012017.xlsx')

In [108]:
vendors = pd.read_excel('vendors.xlsx')

In [109]:
po_lines.columns

Index(['purno', 'vendno', 'vendname', 'custno', 'custpo', 'transfer', 'item',
       'altitem', 'descrip', 'cost', 'qtyord', 'qtyrec', 'qtyrecx', 'qtylft',
       'qtybo', 'extcost', 'seq', 'postat', 'potype', 'custname', 'tranitem',
       'nsell', 'created', 'qtytorec', 'price', 'final', 'type', 'cponum'],
      dtype='object')

In [132]:
vendors.columns

Index(['vendno', 'company', 'contact', 'title', 'address1', 'address2', 'city',
       'state', 'zip', 'country', 'phone', 'pterms', 'pnet', 'comment1',
       'comment2', 'faxno', 'statusnotes', 'check_this', 'commodity',
       'supp_type', 'exp_date', 'app_status', 'app_date', 'updated'],
      dtype='object')

In [143]:
type(vendors['vendno'][1])

numpy.float64

In [144]:
type(po_lines['vendno'][1])

numpy.int64

In [135]:
vendors.vendno = pd.to_numeric(vendors.vendno, errors='coerce')

In [141]:
po_line_with_detail = po_lines.merge(vendors,on='vendno')

In [142]:
po_line_with_detail.columns

Index(['purno', 'vendno', 'vendname', 'custno', 'custpo', 'transfer', 'item',
       'altitem', 'descrip', 'cost', 'qtyord', 'qtyrec', 'qtyrecx', 'qtylft',
       'qtybo', 'extcost', 'seq', 'postat', 'potype', 'custname', 'tranitem',
       'nsell', 'created', 'qtytorec', 'price', 'final', 'type', 'cponum',
       'company', 'contact', 'title', 'address1', 'address2', 'city', 'state',
       'zip', 'country', 'phone', 'pterms', 'pnet', 'comment1', 'comment2',
       'faxno', 'statusnotes', 'check_this', 'commodity', 'supp_type',
       'exp_date', 'app_status', 'app_date', 'updated'],
      dtype='object')

In [151]:
item_origin_info = po_line_with_detail[['item','altitem','descrip','vendno','state','country']]

In [152]:
item_origin_info.head()

,item,altitem,descrip,vendno,state,country
0,415154,415154,Recoil Spring. 0725 X 4.639,304300,CT,NaN
1,415154,415154,Recoil Spring. 0725 X 4.639,304300,CT,NaN
2,TEST REPORTS,TR,TEST REPORTS,304300,CT,NaN
3,300015-PLAIN,300015-PLAIN,NaN,219300,Taiwan ROC,NaN
4,1C600-MFH-40,NaN,NaN,219300,Taiwan ROC,NaN


In [153]:
item_origin_info.country.unique()

array([nan, 'TED EXT 26'], dtype=object)

In [160]:
item_origin_info.state.unique()

array(['CT', 'Taiwan ROC', 'NY', 'IL', 'FL', 'NJ', 'CA', 'California',
       'TX', 'NH', 'PA', 'ID', 'VT', 'Texas', 'WI', 'MN', 'KS', 'OH',
       'China', nan, 'AR', 'GA', 'MA', 'CHINA', 'CA.', 'MI', 'RI', 'AL',
       'Co', 'NV', 'Mn', 'Maine', 'Canada', 'Georgia', '6BX', 'Taiwan',
       'VA', 'Ohio', 'TAIWAN', 'MD', 'MO', 'IN', 'QC', 'ME', 'CO', 'WA',
       'AZ', 'GUANGDONG', 'TAIWAN ROC', 'IL.', 'PA.', 'NC', 'Il',
       'LE4 7ST', 'Illinois', 'OHIO', 'Zhejiang', 'KY', 'Guangdong',
       'ZHEJIANG'], dtype=object)

In [165]:
china_strings = ['GUANGDONG','ZHEJIANG','Zhejiang','Guangdong','China','CHINA']

In [166]:
taiwan_strings = ['TAIWAN ROC','Taiwan ROC','Taiwan','TAIWAN']

In [168]:
item_origin_info['state'].isin(taiwan_strings).head()

0    False
1    False
2    False
3     True
4     True
Name: state, dtype: bool

In [172]:
item_origin_info['coo'] = 'US'

/Users/joshua/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [173]:
item_origin_info['coo'] = ['TW' if region.isin(taiwan_strings) else 'US' for region in item_origin_info['state']]

AttributeError: 'str' object has no attribute 'isin'